In [3]:
from langchain.llms import OpenAI
import os

In [4]:
#add secret key openai
os.environ['OPEN_API_KEY']=""

Temperature Value --> how creative we want our model to be.

0 --> temperature it means model is very safe it is not taking any bets.

1 --> it will take risk it might generate wrong output but it is very creative

In [16]:
llm = OpenAI(openai_api_key=os.environ['OPEN_API_KEY'] ,temperature=0.7)

In [4]:
text = "What is the capital of india"
print(llm.predict(text))



The capital of India is New Delhi.


In [3]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""

In [4]:
from langchain import HuggingFaceHub


In [11]:
llm_hugging =HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0.7,"max_length":64})

t:\LangChain\langchain\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [12]:
output= llm_hugging.predict("Can you write a poem about AI")

In [13]:
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [17]:
llm.predict("Can you write a poem about AI")

'\n\nAI is here to help us find our way \nTo make decisions, diagnosis, and more each day \nIt can sometimes be seen as a saving grace \nBut it’s not too far from taking over the place \n\nIt can be seen as a blessing or a curse \nIt can be used to help us or make our lives worse \nAI can be the gift we could never have dreamed of \nBut it could also be the thing that takes us away from love \n\nIt’s a technology that’s both powerful and scary \nIt could make us more efficient or make us less wary \nBut no matter what, AI will always be here \nSo let’s use it wisely and show it some care.'

## Prompt Templates And Chain

In [19]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'],
template = "Tell me the capital of this {country}")
prompt_template.format(country ="India")


'Tell me the capital of this India'

In [20]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm,prompt=prompt_template)
chain.run("India")

'\n\nThe capital of India is New Delhi.'

## Combining multiple chain using simple Sequential chain

In [21]:
capital_prompt = PromptTemplate(input_variables=['country'],template="Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm,prompt=capital_prompt)

In [22]:
famous_template = PromptTemplate(input_variables=['capital'],template='Suggest me some amazing place to visit in {capital}')
famous_chai = LLMChain(llm=llm,prompt=famous_template)

In [23]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chai])
chain.run("India")

" There are many amazing places to visit in and around the city. Some of the most popular places to visit in New Delhi are:\n\n1. India Gate\n2. Red Fort\n3. Jama Masjid\n4. Lotus Temple\n5. Qutub Minar\n6. Humayun's Tomb\n7. Akshardham Temple\n8. Chandni Chowk\n9. Jantar Mantar\n10. Rashtrapati Bhavan"

Sequential Chain

In [28]:
capital_prompt = PromptTemplate(input_variables=['country'],template="Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm,prompt=capital_prompt,output_key="capital")

In [29]:
famous_template = PromptTemplate(input_variables=['capital'],template='Suggest me some amazing place to visit in {capital}')
famous_chai = LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [30]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain,famous_chai],input_variables=['country'],output_variables=['capital','places'])

In [31]:
chain({'country':'India'})

{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Some amazing places to visit in New Delhi are:\n\n1. Red Fort: A UNESCO World Heritage Site, the majestic Red Fort is a must-visit in Delhi. \n\n2. India Gate: This memorial commemorates the soldiers who lost their lives fighting for the country.\n\n3. Qutub Minar: This UNESCO World Heritage Site is a towering minaret, standing at a height of 73 metres.\n\n4. Humayun's Tomb: This stunning structure is a UNESCO World Heritage Site and is a must-visit for history buffs.\n\n5. Lotus Temple: This stunning temple is a Baháʼí House of Worship and is a must-visit for tourists.\n\n6. Akshardham Temple: This sprawling temple complex is one of the most visited places in Delhi.\n\n7. Jama Masjid: This stunning mosque is one of the largest in India and is a must-visit for tourists.\n\n8. Chandni Chowk: This bustling market is a great place to shop for traditional Indian wares.\n\n9. Rashtrapati Bhavan: This gra

: 

Chatmodels with ChatOpenAI

In [1]:
from langchain.chat_models import ChatOpenAI

In [5]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage
chat_llm = ChatOpenAI(openai_api_key=os.environ['OPEN_API_KEY'] ,temperature=0.7,model='gpt-3.5-turbo')

In [6]:
chat_llm(
[
    SystemMessage(content='Your are a comedian AI assitant'),
    HumanMessage(content="Please provide some comedy punchline on AI")
])

AIMessage(content='Why did the AI go to therapy? Because it had too many unresolved issues with Siri-ous jealousy!')

Prompt Template LLM Output Parsers

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser


In [8]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")

In [12]:
template = "You are a helpful assistant . When the user given any input, you should generate 5 words synonym in a comma separated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [13]:
chain = chatprompt|chat_llm|Commaseperatedoutput()

In [14]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' knowledgeable', ' quick-witted', ' sharp']

: 